## import libraries

In [2]:
from cadv_exploration.utils import load_dotenv

load_dotenv()
from scripts.python.utils import setup_logger
from cadv_exploration.llm.langchain.downstream_task_prompt import SQL_QUERY_TASK_DESCRIPTION, \
    ML_INFERENCE_TASK_DESCRIPTION
from inspector.deequ.deequ_inspector_manager import DeequInspectorManager
from cadv_exploration.llm.langchain import LangChainCADV
from data_models import Constraints

from loader import FileLoader

from cadv_exploration.dq_manager import DeequDataQualityManager
from cadv_exploration.utils import get_project_root
import pandas as pd
import oyaml as yaml

logger = setup_logger("toy_example.log")

/Users/haochen/Library/Caches/pypoetry/virtualenvs/cadv-exploration-4wWqlI_J-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Load the data and utility functions

In [3]:
dq_manager = DeequDataQualityManager()
train_file_path = get_project_root() / "data" / "toy_example" / "files" / "hospitalisations_train.csv"
test_file_path = get_project_root() / "data" / "toy_example" / "files" / "hospitalisations_test.csv"
train_data = FileLoader.load_csv(train_file_path, na_values=["NULL"])
test_data = FileLoader.load_csv(test_file_path, na_values=["NULL"])
spark_train_data, spark_train = dq_manager.spark_df_from_pandas_df(train_data)
spark_test_data, spark_test = dq_manager.spark_df_from_pandas_df(test_data)


def validate_on_test_data(constraints, test_data):
    code_column_map = constraints.get_suggestions_code_column_map(valid_only=False)
    code_list = [item for item in code_column_map.keys()]
    spark_test_data, spark_test = dq_manager.spark_df_from_pandas_df(test_data)
    status_on_test_data = dq_manager.validate_on_spark_df(spark_test, spark_test_data, code_list,
                                                          return_raw=True)
    code_list_for_constraints = [
        (code_list[i], status_on_test_data[i].constraint_status, status_on_test_data[i].constraint_message) for i
        in
        range(len(code_list))]
    return pd.DataFrame(code_list_for_constraints, columns=["column_name", "constraint_status", "constraint_message"])

:: loading settings :: url = jar:file:/Users/haochen/Library/Caches/pypoetry/virtualenvs/cadv-exploration-4wWqlI_J-py3.9/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/haochen/.ivy2/cache
The jars for the packages stored in: /Users/haochen/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-48bbc863-fc3d-445b-ac3b-d3d0a7a630f5;1.0
	confs: [default]
	found com.amazon.deequ#deequ;2.0.7-spark-3.5 in central
	found org.scala-lang#scala-reflect;2.12.10 in local-m2-cache
	found org.scalanlp#breeze_2.12;2.1.0 in local-m2-cache
	found org.scalanlp#breeze-macros_2.12;2.1.0 in local-m2-cache
	found org.typelevel#spire_2.12;0.17.0 in local-m2-cache
	found org.typelevel#spire-macros_2.12;0.17.0 in local-m2-cache
	found org.typelevel#algebra_2.12;2.0.1 in local-m2-cache
	found org.typelevel#cats-kernel_2.12;2.1.1 in local-m2-cache
	found org.typelevel#spire-platform_2.12;0.17.0 in local-m2-cache
	found org.typelevel#spire-util_2.12;0.17.0 in local-m2-cache
	found dev.ludovic.netlib#blas;3.0.1 in local-m2-cache
	found dev.ludovic.netlib#lapack;3.0.1 in local-m2-c

In [4]:
train_data

,ssn,gender,race,bloodtype,diagnosis,admission_day,discharge_day,insurance,cost,complications
0,420-64-XXXX,1,white,A pos,cough,10,10,UHG,10,N
1,423-33-XXXX,1,asian,O pos,fraction,10,12,KP,1000,N
2,545-31-XXXX,2,white,A pos,fraction,17,19,UHG,1000,N
3,222-24-XXXX,1,hispanic,O pos,cancer,20,25,CG,10000,N


In [5]:
test_data

,ssn,gender,race,bloodtype,diagnosis,admission_day,discharge_day,insurance,cost,complications
0,221-04-XXXX,1.0,black,AB neg,stroke,21.0,21,UHG,0,N
1,434-29-XXXX,NaN,asian,O pos,NaN,NaN,22,KP,1000,Y
2,212-56-XXXX,1.0,white,O pos,bloodknot,22.0,29,UHG,1000,N


## get constraints with deequ

In [6]:
constraints = dq_manager.get_constraints_for_spark_df(spark_train, spark_train_data, spark_test,
                                                      spark_test_data)
print(yaml.dump(constraints.to_dict()))

25/03/03 17:32:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Python Callback server started!
constraints:
  admission_day:
    code:
    - - .isComplete("admission_day")
      - Invalid
    - - .isNonNegative("admission_day")
      - Valid
    assumptions: []
  bloodtype:
    code:
    - - .isComplete("bloodtype")
      - Valid
    - - .isContainedIn("bloodtype", ["A pos", "O pos"])
      - Invalid
    assumptions: []
  complications:
    code:
    - - .isComplete("complications")
      - Valid
    - - .isContainedIn("complications", ["N"])
      - Invalid
    assumptions: []
  cost:
    code:
    - - .isComplete("cost")
      - Valid
    - - .isNonNegative("cost")
      - Valid
    assumptions: []
  diagnosis:
    code:
    - - .isComplete("diagnosis")
      - Invalid
    assumptions: []
  discharge_day:
    code:
    - - .isComplete("discharge_day")
      - Valid
    - - .isNonNegative("discharge_day")
      - Valid
    - - .isUnique("discharge_day")
      - Valid
    assumptions: []
  gender:
    code:
    - - .isComplete("gender")
      - In

In [7]:
validate_on_test_data(constraints, test_data)

,column_name,constraint_status,constraint_message
0,".isComplete(""admission_day"")",Failure,Value: 0.6666666666666666 does not meet the co...
1,".isNonNegative(""admission_day"")",Success,
2,".isComplete(""gender"")",Failure,Value: 0.6666666666666666 does not meet the co...
3,".isNonNegative(""gender"")",Success,
4,".isComplete(""race"")",Success,
5,".isContainedIn(""bloodtype"", [""A pos"", ""O pos""])",Failure,Value: 0.6666666666666666 does not meet the co...
6,".isComplete(""bloodtype"")",Success,
7,".isContainedIn(""complications"", [""N""])",Failure,Value: 0.6666666666666666 does not meet the co...
8,".isComplete(""complications"")",Success,
9,".isComplete(""cost"")",Success,


# downstream task 1

## prepare context for LLM

In [8]:
column_desc = DeequInspectorManager().spark_df_to_column_desc(spark_train_data, spark_train)
context = """
nonsensitive_df = duckdb.sql("SELECT * EXCLUDE ssn, gender, race
FROM 's3://datalake/latest/hospitalisations.csv'").df()
hosp_df = nonsensitive_df.dropna()
strokes_total = duckdb.sql("SELECT COUNT(*) FROM hosp_df
WHERE diagnosis = 'stroke'").fetch()
strokes_for_rare_bloodtypes = duckdb.sql("SELECT COUNT(*)
FROM hosp_df WHERE diagnosis = 'stroke'
AND bloodtype IN ('AB negative', 'B negative')").fetch()
generate_report(strokes_total, strokes_for_rare_bloodtypes)"""

## run LLM on toy model with default settings

In [9]:
lc = LangChainCADV(model_name="gpt-4.5-preview", downstream_task_description=SQL_QUERY_TASK_DESCRIPTION,
                   assumption_generation_trick=None, logger=logger)

relevant_columns_list, expectations, suggestions = lc.invoke(
    input_variables={"column_desc": column_desc, "script": context},
    num_stages=3,
    max_retries=3
)
code_list_for_constraints = [item for v in suggestions.values() for item in v]

# Validate the constraints on the original data to see if they are grammarly correct
code_list_for_constraints_valid = dq_manager.filter_constraints(code_list_for_constraints, spark_test,
                                                                spark_test_data)
constraints = Constraints.from_llm_output(relevant_columns_list, expectations, suggestions,
                                          code_list_for_constraints_valid)

print(yaml.dump(constraints.to_dict()))

constraints:
  bloodtype:
    code:
    - - '.hasCompleteness(''bloodtype'', lambda x: x == 1.0)'
      - Valid
    - - '.hasUniqueValueRatio([''bloodtype''], lambda x: x > 0)'
      - Valid
    - - .isComplete('bloodtype')
      - Valid
    - - .isContainedIn('bloodtype', ['AB negative', 'B negative', 'A positive', 'A
        negative', 'B positive', 'AB positive', 'O positive', 'O negative'])
      - Invalid
    assumptions:
    - The column 'bloodtype' should contain values 'AB negative' or 'B negative' if
      analysis of rare blood types is expected.
    - The column 'bloodtype' should not have missing or null values.
  diagnosis:
    code:
    - - .isComplete('diagnosis')
      - Invalid
    - - .isContainedIn('diagnosis', ['stroke'])
      - Invalid
    assumptions:
    - The column 'diagnosis' should contain the value 'stroke' if strokes are expected
      in analysis.
    - The column 'diagnosis' should not have missing or null values.



In [10]:
validate_on_test_data(constraints, test_data)

,column_name,constraint_status,constraint_message
0,.isComplete('diagnosis'),Failure,Value: 0.6666666666666666 does not meet the co...
1,".isContainedIn('diagnosis', ['stroke'])",Failure,Value: 0.6666666666666666 does not meet the co...
2,.isComplete('bloodtype'),Success,
3,".hasCompleteness('bloodtype', lambda x: x == 1.0)",Success,
4,".hasUniqueValueRatio(['bloodtype'], lambda x: ...",Success,
5,".isContainedIn('bloodtype', ['AB negative', 'B...",Failure,Value: 0.0 does not meet the constraint requir...


## run LLM on toy model with with_deequ trick

It will add, delete and modify the constraints generated by deequ

In [11]:
deequ_assumptions = dq_manager.get_constraints_for_spark_df(spark_train, spark_train_data).to_string()
lc = LangChainCADV(model_name="gpt-4.5-preview", downstream_task_description=SQL_QUERY_TASK_DESCRIPTION,
                   assumption_generation_trick='with_deequ', logger=logger)

relevant_columns_list, expectations, suggestions = lc.invoke(
    input_variables={"column_desc": column_desc, "script": context, "deequ_assumptions": deequ_assumptions},
    num_stages=3,
    max_retries=3
)
code_list_for_constraints = [item for v in suggestions.values() for item in v]

# Validate the constraints on the original data to see if they are grammarly correct
code_list_for_constraints_valid = dq_manager.filter_constraints(code_list_for_constraints, spark_test,
                                                                spark_test_data)
constraints = Constraints.from_llm_output(relevant_columns_list, expectations, suggestions,
                                          code_list_for_constraints_valid)

print(yaml.dump(constraints.to_dict()))

constraints:
  bloodtype:
    code:
    - - .isComplete('bloodtype')
      - Valid
    - - '.satisfies("bloodtype IN (''AB negative'', ''B negative'')", ''contains_rare_bloodtypes'',
        lambda x: x > 0)'
      - Invalid
    assumptions:
    - Column 'bloodtype' should not contain null values.
    - Column 'bloodtype' should include 'AB negative' or 'B negative' to ensure the
      query returns meaningful results for rare blood types.
  diagnosis:
    code:
    - - .isComplete('diagnosis')
      - Invalid
    - - .isContainedIn('diagnosis', ['stroke'])
      - Invalid
    assumptions:
    - Column 'diagnosis' should not contain null values.
    - Column 'diagnosis' should include the value 'stroke' to ensure the query returns
      meaningful results.



In [12]:
validate_on_test_data(constraints, test_data)

,column_name,constraint_status,constraint_message
0,.isComplete('diagnosis'),Failure,Value: 0.6666666666666666 does not meet the co...
1,".isContainedIn('diagnosis', ['stroke'])",Failure,Value: 0.6666666666666666 does not meet the co...
2,.isComplete('bloodtype'),Success,
3,".satisfies(""bloodtype IN ('AB negative', 'B ne...",Failure,Value: 0.0 does not meet the constraint requir...


## run LLM on toy model with with_experience trick

In [13]:
lc = LangChainCADV(model_name="gpt-4.5-preview", downstream_task_description=SQL_QUERY_TASK_DESCRIPTION,
                   assumption_generation_trick="with_experience", logger=logger)

relevant_columns_list, expectations, suggestions = lc.invoke(
    input_variables={"column_desc": column_desc, "script": context},
    num_stages=3,
    max_retries=3
)
code_list_for_constraints = [item for v in suggestions.values() for item in v]

# Validate the constraints on the original data to see if they are grammarly correct
code_list_for_constraints_valid = dq_manager.filter_constraints(code_list_for_constraints, spark_test,
                                                                spark_test_data)
constraints = Constraints.from_llm_output(relevant_columns_list, expectations, suggestions,
                                          code_list_for_constraints_valid)

print(yaml.dump(constraints.to_dict()))

constraints:
  bloodtype:
    code:
    - - .isComplete('bloodtype')
      - Valid
    - - .isContainedIn('bloodtype', ['O pos', 'A pos', 'AB negative', 'B negative',
        'O negative', 'A negative', 'AB positive', 'B positive'])
      - Invalid
    - - '.satisfies("bloodtype IN (''AB negative'', ''B negative'')", ''rare_bloodtype_present'',
        lambda x: x > 0)'
      - Invalid
    assumptions:
    - The column 'bloodtype' should not contain NULL values.
    - The column 'bloodtype' should contain values ['O pos', 'A pos', 'AB negative',
      'B negative'] and other valid blood types like ['O negative', 'A negative',
      'AB positive', 'B positive'].
  diagnosis:
    code:
    - - .isComplete('diagnosis')
      - Invalid
    - - .isContainedIn('diagnosis', ['stroke', 'cancer', 'cough', 'fraction'])
      - Invalid
    - - '.satisfies("diagnosis = ''stroke''", ''stroke_present'', lambda x: x > 0)'
      - Valid
    assumptions:
    - The column 'diagnosis' should not contain 

In [14]:
validate_on_test_data(constraints, test_data)

,column_name,constraint_status,constraint_message
0,.isComplete('diagnosis'),Failure,Value: 0.6666666666666666 does not meet the co...
1,".isContainedIn('diagnosis', ['stroke', 'cancer...",Failure,Value: 0.6666666666666666 does not meet the co...
2,".satisfies(""diagnosis = 'stroke'"", 'stroke_pre...",Success,
3,.isComplete('bloodtype'),Success,
4,".isContainedIn('bloodtype', ['O pos', 'A pos',...",Failure,Value: 0.6666666666666666 does not meet the co...
5,".satisfies(""bloodtype IN ('AB negative', 'B ne...",Failure,Value: 0.0 does not meet the constraint requir...


# Downstream task 2

## prepare context for LLM

In [15]:
column_desc = DeequInspectorManager().spark_df_to_column_desc(spark_train_data, spark_train)
context = """
df = pd.read_csv("s3://datalake/latest/hospitalisations.csv") 
df['cost_smoothed'] = np.log(df['cost'])
df['admission_day'].fillna(df['discharge_day'])
df['duration'] = df['discharge_day'] - df['admission_day']
categorical_cols = ['diagnosis', 'insurance']
for col in categorical_cols:
 df[col] = pd.get_dummies(df[col], dummy_na=True)
features = df[categorical_cols + ['duration', 'cost_smoothed']]
labels = label_binarize(df['complications'], classes=['Y', 'N'])
model = sklearn.tree.DecisionTreeClassifier()
model.fit(train_features, train_labels)
deploy_to_production(model)
"""

## run LLM on toy model with default settings

In [16]:
lc = LangChainCADV(model_name="gpt-4.5-preview", downstream_task_description=ML_INFERENCE_TASK_DESCRIPTION,
                   assumption_generation_trick=None, logger=logger)

relevant_columns_list, expectations, suggestions = lc.invoke(
    input_variables={"column_desc": column_desc, "script": context},
    num_stages=3,
    max_retries=3
)
code_list_for_constraints = [item for v in suggestions.values() for item in v]

# Validate the constraints on the original data to see if they are grammarly correct
code_list_for_constraints_valid = dq_manager.filter_constraints(code_list_for_constraints, spark_test,
                                                                spark_test_data)
constraints = Constraints.from_llm_output(relevant_columns_list, expectations, suggestions,
                                          code_list_for_constraints_valid)

print(yaml.dump(constraints.to_dict()))

constraints:
  admission_day:
    code:
    - - '.haveAnyCompleteness([''admission_day'', ''discharge_day''], lambda x: x
        == 1.0)'
      - Valid
    assumptions:
    - Values should not be null or missing, or if missing, discharge_day should be
      available for imputation
  complications:
    code:
    - - .isComplete('complications')
      - Valid
    - - .isContainedIn('complications', ['Y', 'N'])
      - Valid
    assumptions:
    - Values should only include 'Y' or 'N'
  cost:
    code:
    - - .isComplete('cost')
      - Valid
    - - .isPositive('cost')
      - Invalid
    assumptions:
    - Values should be greater than 0, since logarithmic transformation is applied
  diagnosis:
    code:
    - - '.hasApproxCountDistinct(''diagnosis'', lambda x: x <= known_category_count)'
      - Invalid
    - - '.hasDistinctness([''diagnosis''], lambda x: x <= 1.0)'
      - Valid
    - - .isComplete('diagnosis')
      - Invalid
    assumptions:
    - Values should only include known

In [17]:
validate_on_test_data(constraints, test_data)

,column_name,constraint_status,constraint_message
0,.isPositive('cost'),Failure,Value: 0.6666666666666666 does not meet the co...
1,.isComplete('cost'),Success,
2,".haveAnyCompleteness(['admission_day', 'discha...",Success,
3,.isComplete('discharge_day'),Success,
4,".satisfies('discharge_day >= admission_day', '...",Failure,Value: 0.6666666666666666 does not meet the co...
5,.isComplete('diagnosis'),Failure,Value: 0.6666666666666666 does not meet the co...
6,".hasDistinctness(['diagnosis'], lambda x: x <=...",Success,
7,".hasApproxCountDistinct('diagnosis', lambda x:...",Failure,Can't execute the assertion: An exception was ...
8,.isComplete('insurance'),Success,
9,".hasDistinctness(['insurance'], lambda x: x <=...",Success,


## run LLM on toy model with with_deequ trick

It will add, delete and modify the constraints generated by deequ

In [18]:
deequ_assumptions = dq_manager.get_constraints_for_spark_df(spark_train, spark_train_data).to_string()
lc = LangChainCADV(model_name="gpt-4.5-preview", downstream_task_description=ML_INFERENCE_TASK_DESCRIPTION,
                   assumption_generation_trick='with_deequ', logger=logger)

relevant_columns_list, expectations, suggestions = lc.invoke(
    input_variables={"column_desc": column_desc, "script": context, "deequ_assumptions": deequ_assumptions},
    num_stages=3,
    max_retries=3
)
code_list_for_constraints = [item for v in suggestions.values() for item in v]

# Validate the constraints on the original data to see if they are grammarly correct
code_list_for_constraints_valid = dq_manager.filter_constraints(code_list_for_constraints, spark_test,
                                                                spark_test_data)
constraints = Constraints.from_llm_output(relevant_columns_list, expectations, suggestions,
                                          code_list_for_constraints_valid)

print(yaml.dump(constraints.to_dict()))

constraints:
  admission_day:
    code:
    - - '.haveAnyCompleteness([''admission_day'', ''discharge_day''], lambda x: x
        == 1.0)'
      - Valid
    - - .isLessThanOrEqualTo('admission_day', 'discharge_day')
      - Invalid
    - - .isNonNegative('admission_day')
      - Valid
    assumptions:
    - The column 'admission_day' must be complete and contain no missing values OR
      if missing, corresponding 'discharge_day' must be complete to fill the NaN values.
    - The values of 'admission_day' must be non-negative integers.
    - The values of 'admission_day' should be less than or equal to 'discharge_day',
      as duration calculation involves subtraction.
  complications:
    code:
    - - .isComplete('complications')
      - Valid
    - - .isContainedIn('complications', ['Y', 'N'])
      - Valid
    assumptions:
    - The column 'complications' must be complete and contain no missing values.
    - The column 'complications' should only contain values from the expected s

In [19]:
validate_on_test_data(constraints, test_data)

,column_name,constraint_status,constraint_message
0,.isComplete('cost'),Success,
1,.isPositive('cost'),Failure,Value: 0.6666666666666666 does not meet the co...
2,".haveAnyCompleteness(['admission_day', 'discha...",Success,
3,.isNonNegative('admission_day'),Success,
4,".isLessThanOrEqualTo('admission_day', 'dischar...",Failure,Value: 0.6666666666666666 does not meet the co...
5,.isComplete('discharge_day'),Success,
6,.isNonNegative('discharge_day'),Success,
7,".isGreaterThanOrEqualTo('discharge_day', 'admi...",Failure,Value: 0.6666666666666666 does not meet the co...
8,.isComplete('diagnosis'),Failure,Value: 0.6666666666666666 does not meet the co...
9,.isComplete('insurance'),Success,


## run LLM on toy model with with_experience trick

In [20]:
lc = LangChainCADV(model_name="gpt-4.5-preview", downstream_task_description=ML_INFERENCE_TASK_DESCRIPTION,
                   assumption_generation_trick="with_experience", logger=logger)

relevant_columns_list, expectations, suggestions = lc.invoke(
    input_variables={"column_desc": column_desc, "script": context},
    num_stages=3,
    max_retries=3
)
code_list_for_constraints = [item for v in suggestions.values() for item in v]

# Validate the constraints on the original data to see if they are grammarly correct
code_list_for_constraints_valid = dq_manager.filter_constraints(code_list_for_constraints, spark_test,
                                                                spark_test_data)
constraints = Constraints.from_llm_output(relevant_columns_list, expectations, suggestions,
                                          code_list_for_constraints_valid)

print(yaml.dump(constraints.to_dict()))

constraints:
  admission_day:
    code:
    - - .isComplete('admission_day')
      - Invalid
    - - .satisfies('admission_day <= discharge_day', 'admission_day_le_discharge_day')
      - Invalid
    assumptions:
    - Values should be integers representing valid day numbers.
    - Values should not be null; otherwise they will be filled from discharge_day.
    - Values should be less than or equal to discharge_day.
  complications:
    code:
    - - .isComplete('complications')
      - Valid
    - - .isContainedIn('complications', ['Y', 'N'])
      - Valid
    assumptions:
    - Values should belong to the set ['Y', 'N'].
    - Values should not contain nulls.
  cost:
    code:
    - - .isComplete('cost')
      - Valid
    - - .isPositive('cost')
      - Invalid
    assumptions:
    - Values should be strictly positive, as log transformation is applied.
    - Values should be numeric and non-negative.
    - Values should not contain nulls.
  diagnosis:
    code:
    - - .isComplete('d

In [21]:
validate_on_test_data(constraints, test_data)

,column_name,constraint_status,constraint_message
0,.isPositive('cost'),Failure,Value: 0.6666666666666666 does not meet the co...
1,.isComplete('cost'),Success,
2,.isComplete('admission_day'),Failure,Value: 0.6666666666666666 does not meet the co...
3,".satisfies('admission_day <= discharge_day', '...",Failure,Value: 0.6666666666666666 does not meet the co...
4,.isComplete('discharge_day'),Success,
5,".satisfies('discharge_day >= admission_day', '...",Failure,Value: 0.6666666666666666 does not meet the co...
6,.isComplete('diagnosis'),Failure,Value: 0.6666666666666666 does not meet the co...
7,.isComplete('insurance'),Success,
8,.isComplete('complications'),Success,
9,".isContainedIn('complications', ['Y', 'N'])",Success,


## stop spark

In [22]:
spark_train.sparkContext._gateway.shutdown_callback_server()
spark_train.stop()
spark_test.sparkContext._gateway.shutdown_callback_server()
spark_test.stop()